In [33]:
!pip install -q accelerate peft bitsandbytes transformers trl -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.3 MB/s eta 0:00:00


In [2]:
!pip install -q evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# !pip install ai2-olmo  # for OLMo

In [1]:
# Import necessary packages for the fine-tuning process
import torch  # PyTorch library for deep learning
from peft import LoraConfig  # Packages for parameter-efficient fine-tuning (PEFT)
from transformers import (
    AutoModelForCausalLM,  # AutoModel for language modeling tasks
    AutoTokenizer,  # AutoTokenizer for tokenization
    BitsAndBytesConfig,  # Configuration for BitsAndBytes
    TrainingArguments,  # Training arguments for model training
)
from trl import SFTTrainer  # SFTTrainer for supervised fine-tuning

In [2]:
model_name = "NousResearch/Llama-2-7b-hf"

In [3]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [4]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [5]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

In [6]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}


1. **Loading the Dataset:**
   The first step involves loading the preprocessed dataset. This dataset will be used for fine-tuning. Preprocessing might involve reformatting prompts, filtering out low-quality text, and combining multiple datasets if needed.

2. **Configuring BitsAndBytes for 4-bit Quantization:**
   The `BitsAndBytesConfig` is set up to enable 4-bit quantization. This configuration is crucial for reducing the memory usage during fine-tuning.

3. **Loading Llama 2 Model and Tokenizer in 4-bit Precision:**
   The Llama 2 model is loaded with 4-bit precision, which significantly reduces the memory footprint. The corresponding tokenizer is also loaded to preprocess the text data.

4. **Loading Configurations and Initializing SFTTrainer:**
   - The configurations needed for QLoRA, which is a parameter-efficient fine-tuning technique, are loaded.
   - Regular training parameters are set up.
   - The `SFTTrainer` is initialized with all the loaded configurations and parameters. This trainer will manage the supervised fine-tuning process.

5. **Start of Training:**
   After all the necessary components are loaded and configured, the training process begins. The `SFTTrainer` takes care of fine-tuning the Llama 2 model using the specified dataset, configurations, and parameters.
   
  These steps collectively set up the environment for fine-tuning a Llama 2 model with 7 billion parameters in 4-bit precision using the QLoRA technique, thus optimizing for VRAM limitations while maintaining model performance.

In [7]:
texts = [
    {
        # https://github.com/pydantic/pydantic/pull/9111
        "code": """
Pull Request Diff:
```
diff --git a/pydantic/_internal/_fields.py b/pydantic/_internal/_fields.py
index 3f78a920d5..6e5e933061 100644
--- a/pydantic/_internal/_fields.py
+++ b/pydantic/_internal/_fields.py
@@ -177,7 +177,7 @@ def collect_model_fields(  # noqa: C901
             )

         # when building a generic model with `MyModel[int]`, the generic_origin check makes sure we don't get
-        # "... shadows an attribute" errors
+        # "... shadows an attribute" warnings
         generic_origin = getattr(cls, '__pydantic_generic_metadata__', {}).get('origin')
         for base in bases:
             dataclass_fields = {
#  Do not warn about shadowed fields if they are not redefined in a child class
```
""",
        "description": """
        # Change Summary

        Adds another early exit condition when evaluating whether to log a warning message during detection for shadowed fields.

In the case where a field is defined in a parent class, but it has not been defined at all in a child class, it is technically not a shadowed field, and so shouldn't be warned as such.

Note this is very different from the case where a child class does redefine a field but with a narrower type or even defined as the same type but with a different default. Conceptually this is probably ok, but checking for that is quite complex and this PR does not attempt to try. So this is about checking if a field is defined or not defined - if it is, regardless of type or default value, the warning message is still logged.
        """,
    },
    {
        # https://github.com/pydantic/pydantic/pull/9144
        "code": """
        --- a/pydantic/main.py
+++ b/pydantic/main.py
@@ -222,9 +222,12 @@ def model_construct(cls: type[Model], _fields_set: set[str] | None = None, **val
         fields_set = set()

         for name, field in cls.model_fields.items():
-            if field.alias and field.alias in values:
+            if field.alias is not None and field.alias in values:
                 fields_values[name] = values.pop(field.alias)
                 fields_set.add(name)
+            elif field.validation_alias is not None and field.validation_alias in values:
+                fields_values[name] = values.pop(field.validation_alias)
+                fields_set.add(name)
             elif name in values:
                 fields_values[name] = values.pop(name)
                 fields_set.add(name)
        """,
        "description": """
        # Change Summary

        just like you can construct a model using a field alias, this PR fixes constructing a model using validation_alias.
        """,
    },
    {
        "code": """
        diff --git a/pydantic/json_schema.py b/pydantic/json_schema.py
index 9f0ceb3e36..3e63ecc08d 100644
--- a/pydantic/json_schema.py
+++ b/pydantic/json_schema.py
@@ -751,6 +751,8 @@ def literal_schema(self, schema: core_schema.LiteralSchema) -> JsonSchemaValue:
             result['type'] = 'boolean'
         elif types == {list}:
             result['type'] = 'array'
+        elif types == {type(None)}:
+            result['type'] = 'null'
         return result

     def enum_schema(self, schema: core_schema.EnumSchema) -> JsonSchemaValue:
        """,
        "description": """
        # Change Summary

        This PR aims to complete #8944 and #8905 by also handling null types when generating a json-schema from a pydantic model.

For instance, the following model:
```python
from pydantic import BaseModel
from typing import Literal


class Foo(BaseModel):
    bar: Literal["Bar"] = 'Bar'
    baz: Literal[None] = None
    foo: str = 'Foo'
```
leads to:
```
{'properties': {'bar': {'const': 'Bar', 'default': 'Bar', 'enum': ['Bar'], 'title': 'Bar', 'type': 'string'}, 'baz': {'const': None, 'default': None, 'enum': [None], 'title': 'Baz', 'type': 'null'}, 'foo': {'default': 'Foo', 'title': 'Foo', 'type': 'string'}}, 'title': 'Foo', 'type': 'object'}
```
        """,
    },
]

In [8]:
from datasets import Dataset

The SFTTrainer supports popular dataset formats. This allows you to pass the dataset to the trainer without any pre-processing directly. The following formats are supported:

instruction format
```json
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

[dataset-format-support](https://huggingface.co/docs/trl/sft_trainer#dataset-format-support)

In [9]:
processed_texts = [
    {
        "prompt": text["code"].strip(),
        "completion": text["description"].strip(),
    }
    for text in texts
]

In [10]:
train_dataset = Dataset.from_list(processed_texts)
eval_dataset = Dataset.from_list(processed_texts)

In [11]:
# Step 2 :Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [12]:
# Step 3 :Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [13]:
# Step 4 :Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [14]:
# Step 5 :Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [15]:
# Step 6 :Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [16]:
# Step 7 :Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    logging_steps=1,
    evaluation_strategy="epoch",
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    # logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    # report_to="tensorboard"
)

In [17]:
import evaluate
import numpy as np

metrics_name = ["bleu", "rouge", "exact_match"]
for name in metrics_name:
    evaluate.load(name)
metrics = evaluate.combine(metrics_name)
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
exact_match = evaluate.load("exact_match")

In [18]:
# https://github.com/huggingface/trl/issues/862#issuecomment-1896074498


def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        logits = logits[0]
    return logits.argmax(dim=-1)


def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(pred.strip()) for pred in decoded_preds]

    decoded_labels = ["\n".join(label.strip()) for label in decoded_labels]

    return metrics.compute(predictions=decoded_preds, references=decoded_labels)

In [19]:
# todo predict only suffix

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)
# todo maybe attn_implementation

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [20]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 121.06 MiB is free. Process 75899 has 14.63 GiB memory in use. Of the allocated memory 14.19 GiB is allocated by PyTorch, and 325.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)